In [15]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import  Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np

In [16]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 20

# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [17]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [18]:

# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=5,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)

Found 1174 images belonging to 14 classes.
Found 428 images belonging to 14 classes.


In [19]:

# Create the classification head
model = Sequential([
   # Convolution Layer 1
    Conv2D(32, (3, 3), input_shape=(img_height,img_width,3), activation='relu'),
    # MaxPooling2D(pool_size=(2, 2)),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolution Layer 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolution Layer 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flattening
    Flatten(),

    # Fully Connected Layer 1
    Dense(128, activation='relu'),
    # Dropout(0.5),

    # Fully Connected Layer 2
    Dense(64, activation='relu'),

    # Output Layer
    Dense(14, activation='sigmoid')  
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 14, 14, 128)     

In [20]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)




Epoch 1/15
59/59 [==============================] - 18s 288ms/step - loss: 2.5494 - accuracy: 0.1482 - val_loss: 2.1926 - val_accuracy: 0.1986
Epoch 2/15
59/59 [==============================] - 15s 261ms/step - loss: 2.0091 - accuracy: 0.3101 - val_loss: 1.7800 - val_accuracy: 0.3294
Epoch 3/15
59/59 [==============================] - 17s 284ms/step - loss: 1.7013 - accuracy: 0.3799 - val_loss: 1.7729 - val_accuracy: 0.3224
Epoch 4/15
59/59 [==============================] - 16s 264ms/step - loss: 1.6432 - accuracy: 0.4114 - val_loss: 1.5963 - val_accuracy: 0.4112
Epoch 5/15
59/59 [==============================] - 16s 273ms/step - loss: 1.4274 - accuracy: 0.4932 - val_loss: 1.6768 - val_accuracy: 0.3551
Epoch 6/15
59/59 [==============================] - 20s 332ms/step - loss: 1.2874 - accuracy: 0.5375 - val_loss: 1.6115 - val_accuracy: 0.3808
Epoch 7/15
59/59 [==============================] - 16s 274ms/step - loss: 1.1549 - accuracy: 0.5775 - val_loss: 1.4068 - val_accuracy: 0.4486

In [21]:
# predicting train dataset 
test_pridiction = model.predict(test_generator)

# true labels 
test_true_labels = test_generator.classes




86/86 [==============================] - 2s 18ms/step


In [22]:
print( test_pridiction.shape)
print( test_true_labels.shape)




(428, 14)
(428,)


In [23]:
#calculating accuracy
accuracy = accuracy_score(test_true_labels, np.argmax(test_pridiction, axis=1))
print(accuracy)

0.6799065420560748
